In [248]:
""" This script tries only using the TuLiP toolbox to solve a multirobot path planning (MrPP) problem.
As opposed to multirobot path execution (MrPE) problem, predefined paths are not given
"""

# Import the packages that we need
from __future__ import print_function
import numpy as np
from tulip import spec
from tulip.transys import machines
from tulip import synth
import gr1_fragment

In [249]:
# Define the parameters & desired trajectories
num_robots = 2  # must be > 1
num_cells = 8
init = np.array([7, 8])
dest = np.array([4, 1])

In [250]:
# Define the env_vars based on the params % trajectories
env_vars = set()
env_vars_list = []
for r in range(1, num_robots + 1):
    env_vars_list.append([])
    for p in range(1, num_cells + 1):
        new_env_var = 's' + str(r) + 'a' + str(p)
        env_vars |= {new_env_var}
        env_vars_list[r - 1].append(new_env_var)

In [251]:
print(env_vars)

{'s2a8', 's1a4', 's1a5', 's2a2', 's1a2', 's2a5', 's2a3', 's2a7', 's1a6', 's2a1', 's1a3', 's2a4', 's2a6', 's1a7', 's1a8', 's1a1'}


In [252]:
print(env_vars_list)

[['s1a1', 's1a2', 's1a3', 's1a4', 's1a5', 's1a6', 's1a7', 's1a8'], ['s2a1', 's2a2', 's2a3', 's2a4', 's2a5', 's2a6', 's2a7', 's2a8']]


In [253]:
# Define the initial conditions
env_init = set()
for r in range(num_robots):
    env_init |= {env_vars_list[r][init[r] - 1]}

In [254]:
print(env_init)

{'s1a7', 's2a8'}


In [255]:
env_safe = set()
env_prog = set()
# Encode the constraint that each robot can only appear in one position
env_unique = set()
for r in range(1, num_robots + 1):
    for i in range(len(env_vars_list[r - 1])):
        new_env_unique = '( ' + env_vars_list[r - 1][i] + ' ) <-> ( '
        for j in range(len(env_vars_list[r - 1])):
            if j != i:
                new_env_unique += '!' + env_vars_list[r - 1][j] + ' && '
        new_env_unique = new_env_unique[:-3] + ')'
        env_unique |= {new_env_unique}
env_safe |= env_unique

In [256]:
print(env_unique)

{'( s2a1 ) <-> ( !s2a2 && !s2a3 && !s2a4 && !s2a5 && !s2a6 && !s2a7 && !s2a8 )', '( s2a5 ) <-> ( !s2a1 && !s2a2 && !s2a3 && !s2a4 && !s2a6 && !s2a7 && !s2a8 )', '( s1a3 ) <-> ( !s1a1 && !s1a2 && !s1a4 && !s1a5 && !s1a6 && !s1a7 && !s1a8 )', '( s2a2 ) <-> ( !s2a1 && !s2a3 && !s2a4 && !s2a5 && !s2a6 && !s2a7 && !s2a8 )', '( s2a7 ) <-> ( !s2a1 && !s2a2 && !s2a3 && !s2a4 && !s2a5 && !s2a6 && !s2a8 )', '( s1a2 ) <-> ( !s1a1 && !s1a3 && !s1a4 && !s1a5 && !s1a6 && !s1a7 && !s1a8 )', '( s1a1 ) <-> ( !s1a2 && !s1a3 && !s1a4 && !s1a5 && !s1a6 && !s1a7 && !s1a8 )', '( s1a6 ) <-> ( !s1a1 && !s1a2 && !s1a3 && !s1a4 && !s1a5 && !s1a7 && !s1a8 )', '( s1a5 ) <-> ( !s1a1 && !s1a2 && !s1a3 && !s1a4 && !s1a6 && !s1a7 && !s1a8 )', '( s2a3 ) <-> ( !s2a1 && !s2a2 && !s2a4 && !s2a5 && !s2a6 && !s2a7 && !s2a8 )', '( s2a4 ) <-> ( !s2a1 && !s2a2 && !s2a3 && !s2a5 && !s2a6 && !s2a7 && !s2a8 )', '( s2a8 ) <-> ( !s2a1 && !s2a2 && !s2a3 && !s2a4 && !s2a5 && !s2a6 && !s2a7 )', '( s1a7 ) <-> ( !s1a1 && !s1a2 && !s1a3

In [257]:
# Encode the transition system relations
transition = np.array([[0, 2, 7, 5], [1, 3, 0, 0], [2, 4, 0, 6], [3, 0, 0, 8], [0, 0, 1, 0], [0, 0, 3, 0], [0, 0, 0, 1], [0, 0, 4, 0]])
# transition = np.array([[0, 2, 0, 0], [1, 0, 0, 0]])
# transition = np.array([[0, 2, 0, 0], [1, 3, 0, 0], [2, 0, 0, 0]])
actions = ['up', 'down', 'left', 'right']
env_trans = set()
sys_trans = set()
# env_trans |= {'( s1a1 ) -> ( stop1 || down1 )'}
# env_trans |= {'( s1a2 ) -> ( stop1 || up1 )'}
for i in range(1, num_cells + 1):
    for r in range(num_robots):
        for d in range(4):
            if transition[i - 1][d] > 0:
                env_trans |= {'( ' + env_vars_list[r][i - 1] + ' && ' + actions[d] + str(r + 1) + ' ) -> X( ' +
                            env_vars_list[r][i - 1] + ' || ' + env_vars_list[r][transition[i - 1][d] - 1] + ' )'}
                # env_trans |= {'( ' + env_vars_list[r][i - 1] + ' && ' + actions[d] + str(r + 1) + ' ) -> X( ' +
                            # env_vars_list[r][transition[i - 1][d] - 1] + ' )'}
            else:
                sys_trans |= {'( ' + env_vars_list[r][i - 1] + ' ) -> ( !' + actions[d] + str(r + 1) + ' )'}
                env_trans |= {'( ' + env_vars_list[r][i - 1] + ' && ' + actions[d] + str(r + 1) + ' ) -> X( ' +
                      env_vars_list[r][i - 1] + ' )'}
        env_trans |= {'( ' + env_vars_list[r][i - 1] + ' && ' + 'stop' + str(r + 1) + ' ) -> X( ' +
                      env_vars_list[r][i - 1] + ' )'}

env_safe |= env_trans

In [258]:
print(env_trans)

{'( s2a8 && stop2 ) -> X( s2a8 )', '( s1a1 && down1 ) -> X( s1a1 || s1a2 )', '( s1a1 && up1 ) -> X( s1a1 )', '( s1a3 && stop1 ) -> X( s1a3 )', '( s1a8 && right1 ) -> X( s1a8 )', '( s1a6 && down1 ) -> X( s1a6 )', '( s1a8 && down1 ) -> X( s1a8 )', '( s1a2 && stop1 ) -> X( s1a2 )', '( s2a1 && right2 ) -> X( s2a1 || s2a5 )', '( s1a2 && down1 ) -> X( s1a2 || s1a3 )', '( s2a3 && right2 ) -> X( s2a3 || s2a6 )', '( s1a6 && right1 ) -> X( s1a6 )', '( s2a6 && right2 ) -> X( s2a6 )', '( s2a4 && stop2 ) -> X( s2a4 )', '( s1a3 && right1 ) -> X( s1a3 || s1a6 )', '( s2a4 && right2 ) -> X( s2a4 || s2a8 )', '( s1a5 && down1 ) -> X( s1a5 )', '( s1a5 && up1 ) -> X( s1a5 )', '( s1a2 && up1 ) -> X( s1a2 || s1a1 )', '( s2a3 && left2 ) -> X( s2a3 )', '( s1a5 && right1 ) -> X( s1a5 )', '( s1a2 && left1 ) -> X( s1a2 )', '( s1a6 && left1 ) -> X( s1a6 || s1a3 )', '( s2a2 && down2 ) -> X( s2a2 || s2a3 )', '( s2a2 && stop2 ) -> X( s2a2 )', '( s2a6 && down2 ) -> X( s2a6 )', '( s2a6 && left2 ) -> X( s2a6 || s2a3 )',

In [259]:
print(len(env_trans))

80


In [260]:
# Encode the constraint that every robot should stay after reaching its destination
sys_dest_stop = set()
sys_dest = set()
for r in range(num_robots):
    sys_dest_stop |= {'( ' + env_vars_list[r][dest[r] - 1] + ' ) -> X( ' + env_vars_list[r][dest[r] - 1] + ' )'}
    sys_dest |= {env_vars_list[r][dest[r] - 1]}
#env_safe |= env_dest_stop

In [261]:
print(sys_dest_stop)

{'( s1a4 ) -> X( s1a4 )', '( s2a1 ) -> X( s2a1 )'}


In [262]:
print(sys_dest)

{'s2a1', 's1a4'}


In [263]:
# Encode the constraint that each robot should either not go or eventually not in the current cell
env_go = set()
for r in range(1, num_robots + 1):
    for i in range(num_cells):
        if i != dest[r - 1] - 1:
            env_go |= {'( !' + env_vars_list[r - 1][i] + ' || ' + 'stop' + str(r) + ' )'}
env_prog |= env_go

# given that I defined the  effect of "unavailable actions" to be staying in place, either we need 
# to update this progress property (because if i am at a cell where left is not possible and i keep
# saying left this progress property requires me to move eventually  which does not make sense)
# the other option is  to explicitly ban the "unavailable actions" in system safety spec

In [264]:
print(env_go)

{'( !s1a7 || stop1 )', '( !s1a3 || stop1 )', '( !s1a8 || stop1 )', '( !s2a3 || stop2 )', '( !s2a7 || stop2 )', '( !s2a8 || stop2 )', '( !s1a5 || stop1 )', '( !s1a1 || stop1 )', '( !s2a4 || stop2 )', '( !s2a5 || stop2 )', '( !s1a2 || stop1 )', '( !s2a2 || stop2 )', '( !s1a6 || stop1 )', '( !s2a6 || stop2 )'}


In [265]:
# Define sys_vars
sys_vars = set()
for r in range(1, num_robots + 1):
    sys_vars |= {'stop' + str(r)}
    sys_vars |= {'left' + str(r)}
    sys_vars |= {'right' + str(r)}
    sys_vars |= {'up' + str(r)}
    sys_vars |= {'down' + str(r)}

In [266]:
print(sys_vars)

{'right2', 'stop2', 'right1', 'stop1', 'up1', 'up2', 'left1', 'left2', 'down2', 'down1'}


In [267]:
# Define initial command. Assume nothing.
sys_init = set()

sys_safe = set()
sys_prog = set()
sys_prog |= sys_dest
sys_safe |= sys_dest_stop
sys_safe |= sys_trans
# Encode the constraint that each time one action must be selected
sys_act = set()
for r in range(1, num_robots + 1):
    sys_act |= {'( left' + str(r) + ' ) <-> ( ' + '!right' + str(r) + ' && ' + '!up' + str(r) + ' && ' + '!down' +
                str(r) + ' && ' + '!stop' + str(r) + ' )'}
    sys_act |= {'( right' + str(r) + ' ) <-> ( ' + '!left' + str(r) + ' && ' + '!up' + str(r) + ' && ' + '!down' +
                str(r) + ' && ' + '!stop' + str(r) + ' )'}
    sys_act |= {'( up' + str(r) + ' ) <-> ( ' + '!right' + str(r) + ' && ' + '!left' + str(r) + ' && ' + '!down' +
                str(r) + ' && ' + '!stop' + str(r) + ' )'}
    sys_act |= {'( down' + str(r) + ' ) <-> ( ' + '!right' + str(r) + ' && ' + '!up' + str(r) + ' && ' + '!left' +
                str(r) + ' && ' + '!stop' + str(r) + ' )'}
    sys_act |= {'( stop' + str(r) + ' ) <-> ( ' + '!right' + str(r) + ' && ' + '!up' + str(r) + ' && ' + '!down' +
                str(r) + ' && ' + '!left' + str(r) + ' )'}
    
sys_safe |= sys_act
print(sys_act)

{'( stop1 ) <-> ( !right1 && !up1 && !down1 && !left1 )', '( left2 ) <-> ( !right2 && !up2 && !down2 && !stop2 )', '( right1 ) <-> ( !left1 && !up1 && !down1 && !stop1 )', '( up1 ) <-> ( !right1 && !left1 && !down1 && !stop1 )', '( down1 ) <-> ( !right1 && !up1 && !left1 && !stop1 )', '( right2 ) <-> ( !left2 && !up2 && !down2 && !stop2 )', '( down2 ) <-> ( !right2 && !up2 && !left2 && !stop2 )', '( stop2 ) <-> ( !right2 && !up2 && !down2 && !left2 )', '( left1 ) <-> ( !right1 && !up1 && !down1 && !stop1 )', '( up2 ) <-> ( !right2 && !left2 && !down2 && !stop2 )'}


In [268]:
# Encode the constraint that if ending point is reached, not go. And if ending point is not reached, always exist some
# robot that goes
sys_stop = set()
new_sys_nonstop = '( '
for r in range(1, num_robots + 1):
    sys_stop |= {'( ' + env_vars_list[r - 1][dest[r - 1] - 1] + ' ) -> ( ' + 'stop' + str(r) + ' )'}
    new_sys_nonstop += '( !' + env_vars_list[r - 1][dest[r - 1] - 1] + ' ) || '
new_sys_nonstop = new_sys_nonstop[:-3] + ') -> ( '
for r in range(1, num_robots + 1):
    new_sys_nonstop += '!stop' + str(r) + ' || '
new_sys_nonstop = new_sys_nonstop[:-3] + ')'
sys_stop |= {new_sys_nonstop}
sys_safe |= sys_stop

In [269]:
print(sys_stop)

{'( ( !s1a4 ) || ( !s2a1 ) ) -> ( !stop1 || !stop2 )', '( s2a1 ) -> ( stop2 )', '( s1a4 ) -> ( stop1 )'}


In [270]:
# Encode the collision avoidance constraints
# No need if only one robot
sys_collision = set()
for i in range(num_cells):
    new_sys_collision = '!( '
    for r in range(num_robots):
        new_sys_collision += env_vars_list[r][i] + ' && '
    new_sys_collision = new_sys_collision[:-3] + ')'
    sys_collision |= {new_sys_collision}
sys_safe |= sys_collision

In [271]:
print(sys_collision)

{'!( s1a2 && s2a2 )', '!( s1a7 && s2a7 )', '!( s1a6 && s2a6 )', '!( s1a5 && s2a5 )', '!( s1a8 && s2a8 )', '!( s1a4 && s2a4 )', '!( s1a3 && s2a3 )', '!( s1a1 && s2a1 )'}


In [272]:
# Create a GR(1) specification
specs = spec.GRSpec(env_vars, sys_vars, env_init, sys_init,
                    env_safe, sys_safe, env_prog, sys_prog)
specs.qinit = '\A \E'  # Moore initial condition synthesized too
specs.moore = False
specs.plus_one = False

In [273]:
print(specs)

((s1a7) && (s2a8) && [](( s1a1 && up1 ) -> X( s1a1 )) && [](( s1a8 && right1 ) -> X( s1a8 )) && [](( s1a6 && down1 ) -> X( s1a6 )) && [](( s2a3 ) <-> ( !s2a1 && !s2a2 && !s2a4 && !s2a5 && !s2a6 && !s2a7 && !s2a8 )) && [](( s2a3 && right2 ) -> X( s2a3 || s2a6 )) && [](( s2a7 && down2 ) -> X( s2a7 )) && [](( s2a4 && stop2 ) -> X( s2a4 )) && [](( s1a3 && right1 ) -> X( s1a3 || s1a6 )) && [](( s2a4 && right2 ) -> X( s2a4 || s2a8 )) && [](( s1a8 && stop1 ) -> X( s1a8 )) && [](( s2a6 && left2 ) -> X( s2a6 || s2a3 )) && [](( s1a4 ) <-> ( !s1a1 && !s1a2 && !s1a3 && !s1a5 && !s1a6 && !s1a7 && !s1a8 )) && [](( s2a6 ) <-> ( !s2a1 && !s2a2 && !s2a3 && !s2a4 && !s2a5 && !s2a7 && !s2a8 )) && [](( s1a2 && up1 ) -> X( s1a2 || s1a1 )) && [](( s2a3 && left2 ) -> X( s2a3 )) && [](( s2a2 ) <-> ( !s2a1 && !s2a3 && !s2a4 && !s2a5 && !s2a6 && !s2a7 && !s2a8 )) && [](( s1a5 && right1 ) -> X( s1a5 )) && [](( s2a2 && down2 ) -> X( s2a2 || s2a3 )) && [](( s2a2 && stop2 ) -> X( s2a2 )) && [](( s2a6 && down2 ) -> 

In [274]:
print('Start synthesis')
ctrl = synth.synthesize(specs)
print('End synthesis')
assert ctrl is not None, 'unrealizable'

Start synthesis
removed 86640 nodes from 86657 total
End synthesis


In [275]:
machines.random_run(ctrl, N=30)

move from
	 state: Sinit
	 with input:{'s2a8': True, 's1a4': False, 's1a5': False, 's2a2': False, 's1a2': False, 's2a5': False, 's2a3': False, 's2a7': False, 's1a6': False, 's2a1': False, 's1a3': False, 's2a4': False, 's2a6': False, 's1a7': True, 's1a8': False, 's1a1': False}
	 to state: 0
	 reacting by producing output: {'right2': False, 'stop2': False, 'right1': False, 'stop1': True, 'up1': False, 'up2': False, 'left1': False, 'left2': True, 'down2': False, 'down1': False}
move from
	 state: 0
	 with input:{'s2a8': True, 's1a4': False, 's1a5': False, 's2a2': False, 's1a2': False, 's2a5': False, 's2a3': False, 's2a7': False, 's1a6': False, 's2a1': False, 's1a3': False, 's2a4': False, 's2a6': False, 's1a7': True, 's1a8': False, 's1a1': False}
	 to state: 0
	 reacting by producing output: {'right2': False, 'stop2': False, 'right1': False, 'stop1': True, 'up1': False, 'up2': False, 'left1': False, 'left2': True, 'down2': False, 'down1': False}
move from
	 state: 0
	 with input:{'s2a8': F

([0,
  0,
  16384,
  16384,
  16384,
  16384,
  16384,
  16384,
  16384,
  24576,
  24576,
  24576,
  32768,
  32768,
  36864,
  36864,
  38912,
  39936,
  39936,
  39936,
  39936,
  46080,
  49152,
  57984,
  55296,
  57984,
  55296,
  57984,
  55296,
  57984],
 {'right2': [False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False],
  'stop2': [False,
   False,
   True,
   True,
   True,
   True,
   True,
   True,
   True,
   True,
   True,
   True,
   True,
   True,
   True,
   True,
   False,
   False,
   False,
   False,
   False,
   False,
   True,
   True,
   True,
   True,
   True,
   True,
   True,
   True],
  'right1': [False,
   False,
   True,
   True,
   True,
   True,
   True,
   True,
   True,
   False,
   False,
   False,
   False,
   Fa

In [29]:
print(len(env_unique))

3
